# Parse Experimental Results & Generate Latex Table

In [1]:
import os, pickle
import numpy as np
import pandas as pd

In [13]:
data_dir = 'data/data-recsys16'

In [14]:
suffix = ['Osak', 'Glas', 'Edin', 'Toro', 'Melb']
datnames = ['Osaka', 'Glasgow', 'Edinburgh', 'Toronto', 'Melbourne']

In [15]:
noshort = False

In [16]:
KX = 100  # 100 folds in user specific setting
kxstr = str(KX) + 'X-'
ALPHA = 0.5
alphastr = str(ALPHA).replace('.', '_') + '-'

Compute the F1 score for recommended trajectory.

In [17]:
def calc_F1(seq_act, seq_rec):
    '''Compute recall, precision and F1 when trajectories contain sub-tours'''
    assert(len(seq_act) > 0)
    assert(len(seq_rec) > 0)
    match_tags = np.zeros(len(seq_act), dtype=np.bool)
    for poi in seq_rec:
        for j in range(len(seq_act)):
            if match_tags[j] == False and poi == seq_act[j]:
                match_tags[j] = True
                break
    intersize = np.nonzero(match_tags)[0].shape[0]
    recall = intersize / len(seq_act)
    precision = intersize / len(seq_rec)
    F1 = 2 * precision * recall / (precision + recall)
    return F1

Load results data.

In [43]:
def load_results(datnames, suffix, dat_ix, noshort, kxstr, alphastr):
    assert(0 <= dat_ix <= len(suffix))
    assert(len(datnames) == len(suffix))
    
    if noshort == True:
        # user specific results
        frecdict_rank_spec = os.path.join(data_dir, 'rank-noshort-specific-' + kxstr + suffix[dat_ix] + '.pkl')
        frecdict_tran_spec = os.path.join(data_dir, 'tran-noshort-specific-' + kxstr + suffix[dat_ix] + '.pkl')
        frecdict_comb_spec = os.path.join(data_dir, 'comb-noshort-specific-' + alphastr + kxstr + suffix[dat_ix]+'.pkl')

        # user agnostic results
        frecdict_rank_agno = os.path.join(data_dir, 'rank-noshort-agnostic-' + suffix[dat_ix] + '.pkl')
        frecdict_tran_agno = os.path.join(data_dir, 'tran-noshort-agnostic-' + suffix[dat_ix] + '.pkl')
        frecdict_comb_agno = os.path.join(data_dir, 'comb-noshort-agnostic-' + alphastr + suffix[dat_ix] + '.pkl')
    else:
        # user specific results
        frecdict_rank_spec = os.path.join(data_dir, 'rank-all-specific-' + kxstr + suffix[dat_ix] + '.pkl')
        frecdict_tran_spec = os.path.join(data_dir, 'tran-all-specific-' + kxstr + suffix[dat_ix] + '.pkl')
        frecdict_comb_spec = os.path.join(data_dir, 'comb-all-specific-' + alphastr + kxstr + suffix[dat_ix] + '.pkl')

        # user agnostic results
        frecdict_rank_agno = os.path.join(data_dir, 'rank-all-agnostic-' + suffix[dat_ix] + '.pkl')
        frecdict_tran_agno = os.path.join(data_dir, 'tran-all-agnostic-' + suffix[dat_ix] + '.pkl')
        frecdict_comb_agno = os.path.join(data_dir, 'comb-all-agnostic-' + alphastr + suffix[dat_ix] + '.pkl')
    
    # load results data
    recdict_rank_spec = pickle.load(open(frecdict_rank_spec, 'rb'))
    recdict_rank_agno = pickle.load(open(frecdict_rank_agno, 'rb'))
    recdict_tran_spec = pickle.load(open(frecdict_tran_spec, 'rb'))
    recdict_tran_agno = pickle.load(open(frecdict_tran_agno, 'rb'))
    recdict_comb_spec = pickle.load(open(frecdict_comb_spec, 'rb'))
    recdict_comb_agno = pickle.load(open(frecdict_comb_agno, 'rb'))
    
    # compute F1
    F1_rank1_spec = []  # rank pop
    F1_rank1_agno = []  # rank pop
    F1_rank2_spec = []  # rank feature
    F1_rank2_agno = []  # rank feature
    for key in sorted(recdict_rank_spec.keys()):
        F1_rank1_spec.append(calc_F1(recdict_rank_spec[key]['REAL'], recdict_rank_spec[key]['REC_POP']))
        F1_rank2_spec.append(calc_F1(recdict_rank_spec[key]['REAL'], recdict_rank_spec[key]['REC_FEATURE']))
    for key in sorted(recdict_rank_agno.keys()):
        F1_rank1_agno.append(calc_F1(recdict_rank_agno[key]['REAL'], recdict_rank_agno[key]['REC_POP']))
        F1_rank2_agno.append(calc_F1(recdict_rank_agno[key]['REAL'], recdict_rank_agno[key]['REC_FEATURE']))
        
    F1_tran1_spec = []  # transition DP
    F1_tran1_agno = []  # transition DP
    F1_tran2_spec = []  # transition ILP
    F1_tran2_agno = []  # transition ILP
    for key in sorted(recdict_tran_spec.keys()):
        F1_tran1_spec.append(calc_F1(recdict_tran_spec[key]['REAL'], recdict_tran_spec[key]['REC_DP']))
        F1_tran2_spec.append(calc_F1(recdict_tran_spec[key]['REAL'], recdict_tran_spec[key]['REC_ILP']))
    for key in sorted(recdict_tran_agno.keys()):
        F1_tran1_agno.append(calc_F1(recdict_tran_agno[key]['REAL'], recdict_tran_agno[key]['REC_DP']))
        F1_tran2_agno.append(calc_F1(recdict_tran_agno[key]['REAL'], recdict_tran_agno[key]['REC_ILP']))

    F1_comb1_spec = []  # combine rank and transition DP
    F1_comb1_agno = []  # combine rank and transition DP
    F1_comb2_spec = []  # combine rank and transition ILP
    F1_comb2_agno = []  # combine rank and transition ILP
    for key in sorted(recdict_comb_spec.keys()):
        F1_comb1_spec.append(calc_F1(recdict_comb_spec[key]['REAL'], recdict_comb_spec[key]['REC_DP']))
        F1_comb2_spec.append(calc_F1(recdict_comb_spec[key]['REAL'], recdict_comb_spec[key]['REC_ILP']))
    for key in sorted(recdict_comb_agno.keys()):
        F1_comb1_agno.append(calc_F1(recdict_comb_agno[key]['REAL'], recdict_comb_agno[key]['REC_DP']))
        F1_comb2_agno.append(calc_F1(recdict_comb_agno[key]['REAL'], recdict_comb_agno[key]['REC_ILP']))
    
    # compute mean and std of F1
    F1dat = [F1_rank1_agno, F1_rank2_agno, F1_tran1_agno, F1_tran2_agno, F1_comb1_agno, F1_comb2_agno, \
             F1_rank1_spec, F1_rank2_spec, F1_tran1_spec, F1_tran2_spec, F1_comb1_spec, F1_comb2_spec]
    F1mean = [np.mean(x) for x in F1dat]
    F1std  = [np.std(x) for x in F1dat]
    
    return F1mean, F1std

In [53]:
strs = []
strs.append('\\begin{table*}\n')
strs.append('\\centering\n')
strs.append('\\caption{Experimental Results}\n')
strs.append('\\small\n')
strs.append('\\begin{tabular}{l|' + 2*'c' + '|' + 2*'c' + '} \\hline\n')
strs.append('\\multirow{2}{*}{Method} ')
strs.append('& \\multicolumn{' + str(2) + '}{|c|}{User agnostic} ')
strs.append('& \\multicolumn{' + str(2) + '}{|c}{User specific} \\\\ \\cline{2-' + str(1+2+2) + '}\n')
strs.append('& Osaka & Glasgow & Osaka & Glasgow \\\\ \\hline\n')
    
dat_ix = 0
F1mean0, F1std0 = load_results(datnames, suffix, dat_ix, noshort, kxstr, alphastr)
maxix0 = np.argmax(F1mean0)

dat_ix = 1
F1mean1, F1std1 = load_results(datnames, suffix, dat_ix, noshort, kxstr, alphastr)
maxix1 = np.argmax(F1mean1)

methods = ['RankP', 'RankF', 'MC-DP', 'MC-ILP', 'Prop-DP', 'Prop-ILP']

assert(len(F1mean0) == len(F1mean1))
assert(len(F1mean0) == 2*len(methods))

for j in range(len(methods)):
    strs.append(methods[j] + ' ')
    strs.append('& $')
    if j == maxix0: strs.append('\\mathbf{')
    strs.append('%.3f' % F1mean0[j]); strs.append('\\pm'); strs.append('%.3f' % F1std0[j])
    if j == maxix0: strs.append('}')
    strs.append('$ ')
    
    strs.append('& $')
    if j == maxix1: strs.append('\\mathbf{')
    strs.append('%.3f' % F1mean1[j]); strs.append('\\pm'); strs.append('%.3f' % F1std1[j])
    if j == maxix1: strs.append('}')
    strs.append('$ ') 
    
    strs.append('& $')
    if j+len(methods) == maxix0: strs.append('\\mathbf{')
    strs.append('%.3f' % F1mean0[j+len(methods)]); strs.append('\\pm'); strs.append('%.3f' % F1std0[j+len(methods)])
    if j+len(methods) == maxix0: strs.append('}')
    strs.append('$ ')
    
    strs.append('& $')
    if j+len(methods) == maxix1: strs.append('\\mathbf{')
    strs.append('%.3f' % F1mean1[j+len(methods)]); strs.append('\\pm'); strs.append('%.3f' % F1std1[j+len(methods)])
    if j+len(methods) == maxix1: strs.append('}')
    strs.append('$ ')    
    strs.append('\\\\\n')

strs.append('\\hline\n')
strs.append('\\end{tabular}\n')
strs.append('\\end{table*}\n')

In [54]:
print(''.join(strs))

\begin{table*}
\centering
\caption{Experimental Results}
\small
\begin{tabular}{l|cc|cc} \hline
\multirow{2}{*}{Method} & \multicolumn{2}{|c|}{User agnostic} & \multicolumn{2}{|c}{User specific} \\ \cline{2-5}
& Osaka & Glasgow & Osaka & Glasgow \\ \hline
RankP & $0.611\pm0.163$ & $0.706\pm0.169$ & $0.611\pm0.163$ & $0.706\pm0.169$ \\
RankF & $0.633\pm0.153$ & $\mathbf{0.758\pm0.182}$ & $0.647\pm0.170$ & $0.754\pm0.179$ \\
MC-DP & $0.707\pm0.229$ & $0.708\pm0.184$ & $0.706\pm0.218$ & $0.695\pm0.182$ \\
MC-ILP & $0.670\pm0.209$ & $0.705\pm0.168$ & $0.656\pm0.198$ & $0.691\pm0.177$ \\
Prop-DP & $\mathbf{0.709\pm0.202}$ & $0.745\pm0.180$ & $0.658\pm0.192$ & $0.714\pm0.183$ \\
Prop-ILP & $0.660\pm0.206$ & $0.743\pm0.179$ & $0.635\pm0.190$ & $0.724\pm0.167$ \\
\hline
\end{tabular}
\end{table*}



In [55]:
strs = []
strs.append('\\begin{table*}\n')
strs.append('\\centering\n')
strs.append('\\caption{Experimental Results}\n')
strs.append('\\small\n')
strs.append('\\begin{tabular}{c|' + 6*'c' + '|' + 6*'c' + '} \\hline\n')
strs.append('\\multirow{2}{*}{Dataset} ')
strs.append('& \\multicolumn{' + str(6) + '}{|c|}{User agnostic} ')
strs.append('& \\multicolumn{' + str(6) + '}{|c}{User specific} \\\\ \\cline{2-' + str(1+6+6) + '}\n')
strs.append('& RankP & RankF & MC-DP & MC-ILP & Prop-DP & Prop-ILP\n')
strs.append('& RankP & RankF & MC-DP & MC-ILP & Prop-DP & Prop-ILP \\\\ \\hline\n')
    
for dat_ix in range(len(suffix)-3):
    F1mean, F1std = load_results(datnames, suffix, dat_ix, noshort, kxstr, alphastr)
    maxix = np.argmax(F1mean)
    strs.append(datnames[dat_ix] + ' ')
    for j in range(len(F1mean)):
        strs.append('& $')
        if j == maxix:
            strs.append('\\mathbf{')
        strs.append('%.2f' % F1mean[j]); strs.append('\\pm'); strs.append('%.2f' % F1std[j])
        if j == maxix: 
            strs.append('}')
        strs.append('$ ')
    strs.append('\\\\\n')

strs.append('\\hline\n')
strs.append('\\end{tabular}\n')
strs.append('\\end{table*}\n')

In [56]:
print(''.join(strs))

\begin{table*}
\centering
\caption{Experimental Results}
\small
\begin{tabular}{c|cccccc|cccccc} \hline
\multirow{2}{*}{Dataset} & \multicolumn{6}{|c|}{User agnostic} & \multicolumn{6}{|c}{User specific} \\ \cline{2-13}
& RankP & RankF & MC-DP & MC-ILP & Prop-DP & Prop-ILP
& RankP & RankF & MC-DP & MC-ILP & Prop-DP & Prop-ILP \\ \hline
Osaka & $0.61\pm0.16$ & $0.63\pm0.15$ & $0.71\pm0.23$ & $0.67\pm0.21$ & $\mathbf{0.71\pm0.20}$ & $0.66\pm0.21$ & $0.61\pm0.16$ & $0.65\pm0.17$ & $0.71\pm0.22$ & $0.66\pm0.20$ & $0.66\pm0.19$ & $0.63\pm0.19$ \\
Glasgow & $0.71\pm0.17$ & $\mathbf{0.76\pm0.18}$ & $0.71\pm0.18$ & $0.70\pm0.17$ & $0.75\pm0.18$ & $0.74\pm0.18$ & $0.71\pm0.17$ & $0.75\pm0.18$ & $0.70\pm0.18$ & $0.69\pm0.18$ & $0.71\pm0.18$ & $0.72\pm0.17$ \\
\hline
\end{tabular}
\end{table*}

